To calculate automatically the chunk (C_1, C_2,..,C_n), which fits to total chunk element size for 'auto' (E ), If we suppose that the chunk size is well distributed in the shape(S_1,S_2,...,S_n), it should follow ;

C_1:S_1=C_2:S_2= ... = C_n:S_2

#coef = C_1/S_1 = C_2 / S_2 ....


C_1 x C_2  x...  x C_n = E

E is calculated by array.chunk-size/dtype.itemsize / (multiple of all the non auto chunk)

Thus it can be re-written as;

C_2 x S_1 = C_1  x S_2 
    thus C_2 = C_1 x S_2 / S_1

...

C_n X S_1 = C_1 x S_n 
    thus C_n = C_1 x S_n / S_1

...
C_1** (n)  = E x  S_1** (n) /  (S_1 x S_2x ... S_n) 


C_1 = (E/ (S_1 x S_2x ... S_n) )** (1/2)   * S_1

Once C_1 is calculated, E should be re-evaluated, and solve C_2 ....

BW may be it is already done in dask somewhere?
Also may be this can be solved using numpy (lapack)'s matrix than this way. 




In [15]:
### COPY AND PAST FROM DASK dask/dask/array/core.py  Lines 2474 to 2620 in 

from distributed.utils import parse_bytes
import math
from numbers import Number, Integral
from dask.utils import  factors
import numpy  as np

def auto_chunks(chunks, shape, limit, dtype, previous_chunks=None):
    """ Determine automatic chunks
    This takes in a chunks value that contains ``"auto"`` values in certain
    dimensions and replaces those values with concrete dimension sizes that try
    to get chunks to be of a certain size in bytes, provided by the ``limit=``
    keyword.  If multiple dimensions are marked as ``"auto"`` then they will
    all respond to meet the desired byte limit, trying to respect the aspect
    ratio of their dimensions in ``previous_chunks=``, if given.
    Parameters
    ----------
    chunks: Tuple
        A tuple of either dimensions or tuples of explicit chunk dimensions
        Some entries should be "auto"
    shape: Tuple[int]
    limit: int, str
        The maximum allowable size of a chunk in bytes
    previous_chunks: Tuple[Tuple[int]]
    See also
    --------
    normalize_chunks: for full docstring and parameters
    """
    if previous_chunks is not None:
        previous_chunks = tuple(
            c if isinstance(c, tuple) else (c,) for c in previous_chunks
        )
    chunks = list(chunks)

    autos = {i for i, c in enumerate(chunks) if c == "auto"}
    if not autos:
        return tuple(chunks)

    if limit is None:
        limit = config.get("array.chunk-size")
    if isinstance(limit, str):
        limit = parse_bytes(limit)

    if dtype is None:
        raise TypeError("DType must be known for auto-chunking")

    if dtype.hasobject:
        raise NotImplementedError(
            "Can not use auto rechunking with object dtype. "
            "We are unable to estimate the size in bytes of object data"
        )

    for x in tuple(chunks) + tuple(shape):
        if (
            isinstance(x, Number)
            and np.isnan(x)
            or isinstance(x, tuple)
            and np.isnan(x).any()
        ):
            raise ValueError(
                "Can not perform automatic rechunking with unknown "
                "(nan) chunk sizes.%s" % unknown_chunk_message
            )

    limit = max(1, limit)

    largest_block = np.prod(
        [cs if isinstance(cs, Number) else max(cs) for cs in chunks if cs != "auto"]
    )

    if previous_chunks:
        # Base ideal ratio on the median chunk size of the previous chunks
        result = {a: np.median(previous_chunks[a]) for a in autos}

        ideal_shape = []
        for i, s in enumerate(shape):
            chunk_frequencies = frequencies(previous_chunks[i])
            mode, count = max(chunk_frequencies.items(), key=lambda kv: kv[1])
            if mode > 1 and count >= len(previous_chunks[i]) / 2:
                ideal_shape.append(mode)
            else:
                ideal_shape.append(s)

        # How much larger or smaller the ideal chunk size is relative to what we have now
        multiplier = (
            limit / dtype.itemsize / largest_block / np.prod(list(result.values()))
        )
        last_multiplier = 0
        last_autos = set()

        while (
            multiplier != last_multiplier or autos != last_autos
        ):  # while things change
            last_multiplier = multiplier  # record previous values
            last_autos = set(autos)  # record previous values

            # Expand or contract each of the dimensions appropriately
            for a in sorted(autos):
                proposed = result[a] * multiplier ** (1 / len(autos))
                if proposed > shape[a]:  # we've hit the shape boundary
                    autos.remove(a)
                    largest_block *= shape[a]
                    chunks[a] = shape[a]
                    del result[a]
                else:
                    result[a] = round_to(proposed, ideal_shape[a])

            # recompute how much multiplier we have left, repeat
            multiplier = (
                limit / dtype.itemsize / largest_block / np.prod(list(result.values()))
            )

        for k, v in result.items():
            chunks[k] = v
        return tuple(chunks)

    else:
        size = (limit / dtype.itemsize / largest_block) ** (1 / len(autos))
        small = [i for i in autos if shape[i] < size]
        if small:
            for i in small:
                chunks[i] = (shape[i],)
            return auto_chunks(chunks, shape, limit, dtype)

        for i in autos:
            chunks[i] = round_to(size, shape[i])

        return tuple(chunks)


def round_to(c, s):
    """ Return a chunk dimension that is close to an even multiple or factor
    We want values for c that are nicely aligned with s.
    If c is smaller than s then we want the largest factor of s that is less than the
    desired chunk size, but not less than half, which is too much.  If no such
    factor exists then we just go with the original chunk size and accept an
    uneven chunk at the end.
    If c is larger than s then we want the largest multiple of s that is still
    smaller than c.
    """
    if c <= s:
        try:
            return max(f for f in factors(s) if c / 2 <= f <= c)
        except ValueError:  # no matching factors within factor of two
            return max(1, int(c))
    else:
        return c // s * s




In [14]:
### COPY AND PAST FROM DASK dask/dask/array/core.py  Lines 2474 to 2620 
### and slight change after the last 'else'

def auto_newchunks(chunks, shape, limit, dtype, previous_chunks=None):
    """ Determine automatic chunks
    This takes in a chunks value that contains ``"auto"`` values in certain
    dimensions and replaces those values with concrete dimension sizes that try
    to get chunks to be of a certain size in bytes, provided by the ``limit=``
    keyword.  If multiple dimensions are marked as ``"auto"`` then they will
    all respond to meet the desired byte limit, trying to respect the aspect
    ratio of their dimensions in ``previous_chunks=``, if given.
    Parameters
    ----------
    chunks: Tuple
        A tuple of either dimensions or tuples of explicit chunk dimensions
        Some entries should be "auto"
    shape: Tuple[int]
    limit: int, str
        The maximum allowable size of a chunk in bytes
    previous_chunks: Tuple[Tuple[int]]
    See also
    --------
    normalize_chunks: for full docstring and parameters
    """
    if previous_chunks is not None:
        previous_chunks = tuple(
            c if isinstance(c, tuple) else (c,) for c in previous_chunks
        )
    chunks = list(chunks)

    autos = {i for i, c in enumerate(chunks) if c == "auto"}
    if not autos:
        return tuple(chunks)

    if limit is None:
        limit = config.get("array.chunk-size")
    if isinstance(limit, str):
        limit = parse_bytes(limit)

    if dtype is None:
        raise TypeError("DType must be known for auto-chunking")

    if dtype.hasobject:
        raise NotImplementedError(
            "Can not use auto rechunking with object dtype. "
            "We are unable to estimate the size in bytes of object data"
        )

    for x in tuple(chunks) + tuple(shape):
        if (
            isinstance(x, Number)
            and np.isnan(x)
            or isinstance(x, tuple)
            and np.isnan(x).any()
        ):
            raise ValueError(
                "Can not perform automatic rechunking with unknown "
                "(nan) chunk sizes.%s" % unknown_chunk_message
            )

    limit = max(1, limit)

    largest_block = np.prod(
        [cs if isinstance(cs, Number) else max(cs) for cs in chunks if cs != "auto"]
    )

    if previous_chunks:
        # Base ideal ratio on the median chunk size of the previous chunks
        result = {a: np.median(previous_chunks[a]) for a in autos}

        ideal_shape = []
        for i, s in enumerate(shape):
            chunk_frequencies = frequencies(previous_chunks[i])
            mode, count = max(chunk_frequencies.items(), key=lambda kv: kv[1])
            if mode > 1 and count >= len(previous_chunks[i]) / 2:
                ideal_shape.append(mode)
            else:
                ideal_shape.append(s)

        # How much larger or smaller the ideal chunk size is relative to what we have now
        multiplier = (
            limit / dtype.itemsize / largest_block / np.prod(list(result.values()))
        )
        last_multiplier = 0
        last_autos = set()

        while (
            multiplier != last_multiplier or autos != last_autos
        ):  # while things change
            last_multiplier = multiplier  # record previous values
            last_autos = set(autos)  # record previous values

            # Expand or contract each of the dimensions appropriately
            for a in sorted(autos):
                proposed = result[a] * multiplier ** (1 / len(autos))
                if proposed > shape[a]:  # we've hit the shape boundary
                    autos.remove(a)
                    largest_block *= shape[a]
                    chunks[a] = shape[a]
                    del result[a]
                else:
                    result[a] = round_to(proposed, ideal_shape[a])

            # recompute how much multiplier we have left, repeat
            multiplier = (
                limit / dtype.itemsize / largest_block / np.prod(list(result.values()))
            )

        for k, v in result.items():
            chunks[k] = v
        return tuple(chunks)

    else:
        size = (limit / dtype.itemsize / largest_block) ** (1 / len(autos))
        small = [i for i in autos if shape[i] < size]
        if small:
            for i in small:
                chunks[i] = (shape[i],)
            return auto_chunks(chunks, shape, limit, dtype)

#        for i in autos:
#            chunks[i] = round_to(size, shape[i])



        chunk_block = limit / dtype.itemsize
        print('after small',chunks)

        for i in autos:
            print('in autosi',i,autos,chunks)
            #auto_chunk_block = chunk_block/(np.prod of non auto chunk values)
            autosleft = {i for i, c in enumerate(chunks) if c == "auto"}
            print('autosleft',autosleft)
            nonresolved = chunk_block
            for i, c in enumerate(chunks):
               if c != "auto":
                   nonresolved=nonresolved /c
            print('nonresolved_block',nonresolved)

            coef3= np.prod(shape)/(nonresolved)
            coef=int(coef3**(1/len(autosleft)))
            size=int(shape[i]/coef)
            print('coef,size',coef,size)
            chunks[i] = round_to(size, shape[i])
            print('computed chunk at i',i,chunks[i])

        return tuple(chunks)
            


def round_to(c, s):
    """ Return a chunk dimension that is close to an even multiple or factor
    We want values for c that are nicely aligned with s.
    If c is smaller than s then we want the largest factor of s that is less than the
    desired chunk size, but not less than half, which is too much.  If no such
    factor exists then we just go with the original chunk size and accept an
    uneven chunk at the end.
    If c is larger than s then we want the largest multiple of s that is still
    smaller than c.
    """
    if c <= s:
        try:
            return max(f for f in factors(s) if c / 2 <= f <= c)
        except ValueError:  # no matching factors within factor of two
            return max(1, int(c))
    else:
        return c // s * s



In [13]:
import dask.array as da
from distributed.utils import parse_bytes
import dask
import numpy  as np
#from core_auto_chunks_try import auto_chunks
#from core_auto_chunks import auto_chunks


limit_list=['258MB','124MB','64MB','378MB','512MB']


timesteps=20834
lat=320
lon=384
shape=(timesteps,lon,lat)
chunks=("auto","auto","auto")
previous_chunks=None

for limit in limit_list: 
    print('---test with shape=',shape,'and limit=',limit)
    dask.config.set({"array.chunk-size": limit})
    x =da.random.RandomState(0).standard_normal(shape, chunks=chunks  )
    dtype=x.dtype
    chunkori=auto_chunks(chunks, shape, limit, dtype=dtype, previous_chunks=previous_chunks)
#    chunknew=auto_newchunks(chunks, shape, limit, dtype=dtype, previous_chunks=previous_chunks)


    print('--chunk computed from dask',x.chunksize,'the size is :',np.prod(x.chunksize)*dtype.itemsize,'bytes, thus ratio is:',np.prod(x.chunksize)*dtype.itemsize/parse_bytes(limit))
    print('--chunk computed from dask',chunkori,'the size is :',np.prod(chunkori)*dtype.itemsize,'bytes, thus ratio is:',np.prod(chunkori)*dtype.itemsize/parse_bytes(limit))
 #   print('--chunk computed from dask',chunknew,'the size is :',np.prod(chunknew)*dtype.itemsize,'bytes, thus ratio is:',np.prod(chunknew)*dtype.itemsize/parse_bytes(limit))





---test with shape= (20834, 384, 320) and limit= 258MB
--chunk computed from dask (318, 192, 160) the size is : 78151680 bytes, thus ratio is: 0.30291348837209303
--chunk computed from dask (318, 192, 160) the size is : 78151680 bytes, thus ratio is: 0.30291348837209303
---test with shape= (20834, 384, 320) and limit= 124MB
--chunk computed from dask (249, 192, 160) the size is : 61194240 bytes, thus ratio is: 0.49350193548387095
--chunk computed from dask (249, 192, 160) the size is : 61194240 bytes, thus ratio is: 0.49350193548387095
---test with shape= (20834, 384, 320) and limit= 64MB
--chunk computed from dask (199, 192, 160) the size is : 48906240 bytes, thus ratio is: 0.76416
--chunk computed from dask (199, 192, 160) the size is : 48906240 bytes, thus ratio is: 0.76416
---test with shape= (20834, 384, 320) and limit= 378MB
--chunk computed from dask (384, 384, 320) the size is : 377487360 bytes, thus ratio is: 0.9986438095238095


TypeError: can't multiply sequence by non-int of type 'tuple'